# Dec 15 – Problem 1

In [35]:
with open("inputs/dec15.txt") as f:
    raw_data = f.read()

In [36]:
map = raw_data.split("\n\n")[0]
moves = raw_data.split("\n\n")[1]
width = len(map.split("\n")[0])
pitch = width+1
height = len(map.split("\n"))

In [37]:
map = [c for c in map]
moves = [c for c in moves if c != "\n"]

In [38]:
robot_x = map.index("@") % pitch
robot_y = map.index("@") // pitch

In [39]:
def get_cell(coords: tuple) -> str | None:
    x, y = coords
    if x < 0 or x >= width or y < 0 or y >= height:
        return None
    return map[y * pitch + x]

In [40]:
def put_cell(coords: tuple, cell: str):
    global map
    x, y = coords
    if x < 0 or x >= width or y < 0 or y >= height:
        return
    map[y * pitch + x] = cell

In [41]:
def step(dir: str):
    global robot_x, robot_y
    if dir == "<": dx, dy = -1, 0
    elif dir == "^":  dx, dy = 0, -1
    elif dir == ">": dx, dy = 1, 0
    elif dir == "v": dx, dy = 0, 1

    x, y = robot_x+dx, robot_y+dy
    cell = get_cell((x, y))
    if cell == "#": return
    elif cell == ".":
        put_cell((robot_x, robot_y), ".")
        put_cell((x, y), "@")
        robot_x, robot_y = x, y
    elif cell == 'O':
        ox, oy = x, y
        while ox >= 1 and ox < width-1 and oy >= 1 and oy < height-1:
            if get_cell((ox, oy)) == "#":
                break
            elif get_cell((ox, oy)) == ".":
                put_cell((ox, oy), "O")
                put_cell((x, y), "@")
                put_cell((robot_x, robot_y), ".")
                robot_x, robot_y = x, y
                break
            
            ox += dx
            oy += dy

In [42]:
for move in moves:
    step(move)

In [43]:
sum = 0
for i in range(len(map)):
    if map[i] == "O":
        x, y = i % pitch, i // pitch
        sum += 100 * y + x

print(sum)

1485257


# Dec 15 – Problem 2

In [44]:
map = raw_data.split("\n\n")[0]
width = len(map.split("\n")[0]) * 2
pitch = width+1
height = len(map.split("\n"))

In [45]:
map_ls = []
for c in map:
    if c == "#": map_ls += ["#", "#"]
    elif c == ".": map_ls += [".", "."]
    elif c == "O": map_ls += ["[", "]"]
    elif c == "@": map_ls += ["@", "."]
    elif c == "\n": map_ls += ["\n"]

map = map_ls

In [46]:
robot_x = map.index("@") % pitch
robot_y = map.index("@") // pitch

In [47]:
def moveable(coords: tuple, dir: int) -> bool:
    x, y = coords
    if get_cell(coords) == "]": x -= 1
    nx, ny = x, y+dir

    c1, c2 = get_cell((nx, ny)), get_cell((nx+1, ny))
    if c1 == "." and c2 == ".": return True
    elif c1 == "#" or c2 == "#": return False
    elif c1 == "[" and c2 == "]": return moveable((nx, ny), dir)
    elif c1 == "]" and c2 == "[": return moveable((nx-1, ny), dir) and moveable((nx+1, ny), dir)
    elif c1 == "]" and c2 != "[": return moveable((nx-1, ny), dir)
    elif c2 == "[" and c1 != "]": return moveable((nx+1, ny), dir)

    return False

In [48]:
def push_vert(coords: tuple, dir: int) -> bool:
    if not moveable(coords, dir): return False

    x, y = coords
    if get_cell(coords) == "]": x -= 1
    nx, ny = x, y+dir

    c1, c2 = get_cell((nx, ny)), get_cell((nx+1, ny))
    if c1 == "#" or c2 == "#": return False
    elif c1 == "." and c2 == ".":
        put_cell((nx, ny), "[")
        put_cell((nx+1, ny), "]")
        put_cell((x, y), ".")
        put_cell((x+1, y), ".")
        return True

    elif c1 == "[" and c2 == "]":
        cleared = push_vert((nx, ny), dir)
    elif c1 == "]" and c2 == "[":
        cleared = push_vert((nx-1, ny), dir) and push_vert((nx+1, ny), dir)
    elif c1 == "]" and c2 != "[":
        cleared = push_vert((nx-1, ny), dir)
    elif c2 == "[" and c1 != "]":
        cleared = push_vert((nx+1, ny), dir)

    if cleared: return push_vert(coords, dir)
    return False

In [49]:
def step2(dir: str):
    global robot_x, robot_y
    if dir == "<": dx, dy = -1, 0
    elif dir == "^": dx, dy = 0, -1
    elif dir == ">": dx, dy = 1, 0
    elif dir == "v": dx, dy = 0, 1

    x, y = robot_x+dx, robot_y+dy
    cell = get_cell((x, y))

    if cell == "#": return
    elif cell == ".":
        put_cell((robot_x, robot_y), ".")
        put_cell((x, y), "@")
        robot_x, robot_y = x, y
    elif cell == "[" or cell == "]":
        ox, oy = x, y
        box_len = 0
        while ox >= 1 and ox < width-1 and oy >= 1 and oy < height-1:
            if get_cell((ox, oy)) == "#":
                break
            elif get_cell((ox, oy)) == ".":
                if dx != 0:
                    for i in range(box_len):
                        if i % 2:
                            if dx > 0: put_cell((x + (i+1)*dx, oy), "]")
                            else: put_cell((x + (i+1)*dx, oy), "[")
                        else:
                            if dx > 0: put_cell((x + (i+1)*dx, oy), "[")
                            else: put_cell((x + (i+1)*dx, oy), "]")
                else:
                    if not push_vert((x, y), dy): break

                put_cell((x, y), "@")
                put_cell((robot_x, robot_y), ".")
                robot_x, robot_y = x, y
                break
            
            box_len += 1
            ox += dx
            oy += dy

In [50]:
for move in moves:
    step2(move)

In [51]:
sum = 0
for i in range(len(map)):
    if map[i] == "[":
        x, y = i % pitch, i // pitch
        sum += 100 * y + x

print(sum)

1475512
